In [16]:
start_date = '2016-01-01'
end_date = '2021-06-01'
target = "10d_ret_standscalar"
alpha_name = "Original_Input_TsStand_Concated"
alpha_list = ['open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
            'open_ts_norm_30d',
             'high_ts_norm_30d',
             'low_ts_norm_30d',
             'close_ts_norm_30d',
             'vwap_ts_norm_30d',
             'volume_ts_norm_30d',
             'pct_chg_ts_norm_30d',
             'turnover_ts_norm_30d',
             'free_turnover_ts_norm_30d'
             ]
shift = 3
sequence = 10
LR = [0.001,0.0001,0.00001]
epoch_num = [30,30,30]

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

# config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
# print('Loading the configuration from ' + config_path)
# configs = namespace.load_namespace(config_path)
# FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
# alpha_list = list(FT.features_in_path.keys())[:5]

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [3]:
a = concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)
a

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


100%|██████████| 1315/1315 [00:33<00:00, 39.31it/s]


open      high       low     close  \
timestamp           ticker                                           
2016-01-08 09:30:00 600652  96496.43  96766.88  89735.19  89735.19   
                    600651  72682.99  73611.10  68274.44  68332.45   
                    600653  50881.54  50881.54  46356.50  46557.61   
                    600601  33010.03  33010.03  30009.11  30124.53   
                    600602   6165.74   6216.90   5751.27   5751.27   
...                              ...       ...       ...       ...   
2021-05-31 09:30:00 601106      2.83      2.83      2.80      2.82   
                    601212      2.79      2.83      2.74      2.76   
                    600968      2.68      2.69      2.62      2.63   
                    601518      2.61      2.66      2.61      2.64   
                    601975      2.23      2.24      2.22      2.23   

                                    vwap     volume  pct_chg  turnover  \
timestamp           ticker                                               
2016-01-08 09:30:00 600652  92274.174246   78092.16  -9.9837    1.2663   
                    600651  70596.469951   60492.68  -9.9388    0.7870   
                    600653  48672.314689   61810.00  -9.3933    0.3539   
                    600601  31435.701479  107836.00  -9.5321    0.4913   
                    600602   5974.060079  101636.83 -10.0080    1.1555   
...                                  ...        ...      ...       ...   
2021-05-31 09:30:00 601106      2.814378  146284.06   0.0000    0.2133   
                    601212      2.786107  495697.66  -0.3623    0.6694   
                    600968      2.648524  426846.22  -0.7663    2.2886   
                    601518      2.641600   36043.95   0.8734    0.2669   
                    601975      2.228500  289735.22   0.0000    0.8083   

                            free_turnover  open_ts_norm_30d  high_ts_norm_30d  \
timestamp           ticker                                                      
2016-01-08 09:30:00 600652         1.4020               NaN               NaN   
                    600651         1.0079               NaN               NaN   
                    600653         0.4121               NaN               NaN   
                    600601         0.5561               NaN               NaN   
                    600602         1.9292               NaN               NaN   
...                                   ...               ...               ...   
2021-05-31 09:30:00 601106         0.5905          0.278753         -0.385640   
                    601212         2.3817          0.117871          0.128580   
                    600968         2.2886          1.552181          1.123518   
                    601518         0.8446          0.040293          0.570559   
                    601975         0.9063         -0.699518         -0.814975   

                            low_ts_norm_30d  close_ts_norm_30d  \
timestamp           ticker                                       
2016-01-08 09:30:00 600652              NaN                NaN   
                    600651              NaN                NaN   
                    600653              NaN                NaN   
                    600601              NaN                NaN   
                    600602              NaN                NaN   
...                                     ...                ...   
2021-05-31 09:30:00 601106     1.067332e-13          -0.110359   
                    601212    -1.052972e-01          -0.331456   
                    600968     1.032690e+00           0.691089   
                    601518     6.871375e-01           0.593183   
                    601975    -6.125173e-01          -0.722735   

                            vwap_ts_norm_30d  volume_ts_norm_30d  \
timestamp           ticker                                         
2016-01-08 09:30:00 600652               NaN                 NaN   
                    600651               

# Shift

In [17]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:14<00:00, 119.83it/s]


[27, 24, 21, 18, 15, 12, 9, 6, 3, 0]


100%|██████████| 1259/1259 [02:18<00:00,  9.09it/s]


open_Shift_27  high_Shift_27  low_Shift_27  \
timestamp           ticker                                               
2016-03-29 09:30:00 000001         875.42         878.86        867.68   
                    000005          72.10          75.53         70.90   
                    000006         278.10         293.78        275.22   
                    000008         196.21         196.87        190.48   
                    000009          70.09          73.39         68.96   
...                                   ...            ...           ...   
2021-05-31 09:30:00 688698          19.38          19.38         18.39   
                    688699          86.73          88.48         82.21   
                    688777          84.00          84.94         82.74   
                    688788          97.03         100.95         96.81   
                    688981          56.55          56.80         55.83   

                            close_Shift_27  vwap_Shift_27  volume_Shift_27  \
timestamp           ticker                                                   
2016-03-29 09:30:00 000001          867.68     872.977624        406178.24   
                    000005           71.92      72.721954        465916.91   
                    000006          281.30     285.272612        491784.65   
                    000008          191.58     193.356934        159255.64   
                    000009           69.93      71.129308        884109.34   
...                                    ...            ...              ...   
2021-05-31 09:30:00 688698           18.55      18.761300         16133.62   
                    688699           86.82      84.720700         15825.90   
                    688777           84.00      83.735000          8250.60   
                    688788          100.63      99.009000          3824.63   
                    688981           56.31      56.311900        178983.04   

                            pct_chg_Shift_27  turnover_Shift_27  \
timestamp           ticker                                        
2016-03-29 09:30:00 000001           -0.5911             0.3441   
                    000005           -0.2571             5.0990   
                    000006            0.4571             3.6660   
                    000008           -1.2486             1.0634   
                    000009            0.8560             5.9349   
...                                      ...                ...   
2021-05-31 09:30:00 688698           -2.5735             4.4052   
                    688699           -1.2174             9.3565   
                    688777            0.5506             2.1138   
                    688788            3.5821             2.2997   
                    688981           -1.3144             1.6336   

                            free_turnover_Shift_27  open_ts_norm_30d_Shift_27  \
timestamp           ticker                                                      
2016-03-29 09:30:00 000001                  0.6823                  -0.486862   
                    000005                  6.3868                   0.127293   
                    000006                  5.5727                   0.141856   
                    000008                  2.1684                  -0.342522   
                    000009                  7.2989                  -0.071600   
...                                            ...                        ...   
2021-05-31 09:30:00 688698                  4.4052                   2.026391   
                    688699                  9.3565                   1.397893   
                    688777                  2.1138                   1.187499   
                    688788                  2.2997                  -1.528578   
                    688981                  1.6336                   0.323252   

                            ...  open_ts_norm_30d_Shift_0  \
timestamp           ticker  ...                             
20

# Task Generation

In [18]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
5,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
6,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
7,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
8,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,running,None


,start_date,end_date
0,2019-01-01,2019-06-01
1,2019-06-01,2020-01-01
2,2020-01-01,2020-06-01
3,2020-06-01,2021-01-01
4,2021-01-01,2021-06-01


In [19]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i" % (shift,sequence),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
0,Original_Input_TsStand_Concated_Shift_3_Sequen...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
1,Original_Input_TsStand_Concated_Shift_3_Sequen...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
2,Original_Input_TsStand_Concated_Shift_3_Sequen...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
3,Original_Input_TsStand_Concated_Shift_3_Sequen...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
4,Original_Input_TsStand_Concated_Shift_3_Sequen...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [20]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
5,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
6,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
7,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
8,Original_Input_Moving_7_Shift_5,6,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,running,None


In [21]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)